In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Number of records
num_records = 14900  # Total count (Success: 8150, Fail: 6750)

# Simulate data for each column
np.random.seed(42)  # Ensure reproducibility

trade_id = [f"T{str(i).zfill(5)}" for i in range(1, num_records + 1)]
trade_type = np.random.choice(['Buy', 'Sell'], size=num_records)
instrument_type = np.random.choice(['Equity', 'Bond', 'Derivative'], size=num_records)
trade_value = np.random.lognormal(mean=13, sigma=1, size=num_records)
trade_volume = np.random.randint(1, 100000, size=num_records)  # Increased range for large trades
counterparty_ids = [f"CP{np.random.randint(1, 50)}" for _ in range(num_records)]

# One-to-one mapping of counterparty risk score and failures
unique_counterparties = list(set(counterparty_ids))
counterparty_risk_scores = {cp: np.random.uniform(0, 1) for cp in unique_counterparties}
counterparty_failures = {cp: np.random.randint(0, 10) for cp in unique_counterparties}

counterparty_risk_score = [counterparty_risk_scores[cp] for cp in counterparty_ids]
counterparty_failures = [counterparty_failures[cp] for cp in counterparty_ids]

# ✅ Ensure exact counts: 8150 'Success' and 6750 'Fail'
settlement_status = ['Fail'] * 6750 + ['Success'] * 8150
np.random.shuffle(settlement_status)  # Shuffle to randomize Fail/Success distribution

settlement_duration = np.random.normal(48, 10, size=num_records)
market_volatility = np.random.uniform(0.1, 0.7, size=num_records)  # Increased max range to allow high volatility
liquidity = np.random.uniform(0.01, 0.1, size=num_records)
processing_time = np.random.randint(1, 60, size=num_records)
manual_intervention = np.random.choice([0, 1], size=num_records, p=[0.7, 0.3])
time_to_settle = settlement_duration + np.random.randint(-5, 5, size=num_records)
counterparty_failure_rate = np.array(counterparty_failures) / 100
currencies = np.random.choice(['USD', 'EUR', 'GBP', 'INR', 'JPY'], size=num_records)
settlement_type = np.random.choice(['FoP', 'AoP'], size=num_records)
client_ids = [f"CL{np.random.randint(1, 50)}" for _ in range(num_records)]

# Generate 50 known ISINs
known_isins = [f"INE{str(i).zfill(6)}{np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'))}" for i in range(50)]
isins = np.random.choice(known_isins, size=num_records)

# Ordered failure reasons with decreasing frequency
failure_reasons_ordered = [
    'Account Difference', 'Invalid Counterparty', 'Currency Difference', 
    'Quantity Difference', 'Documentation Issue',  
    'Client Instruction Missing', 'Issues at the CSD'
]

# Normalize weights to sum to 1
failure_weights = np.array([0.20, 0.15, 0.12, 0.11, 0.10, 0.09, 0.08])
failure_weights /= failure_weights.sum()  # Normalize

# Identify conditions for assigning failure reasons
is_high_trade_volume = trade_volume > 80000

# Assign settlement failure reasons based on conditions
settlement_failure_reason = [None] * num_records

for i in range(num_records):
    if settlement_status[i] == 'Fail':
        if is_high_trade_volume[i]:  # Ensure these reasons appear only when trade_volume > 80,000
            if trade_type[i] == 'Buy':
                settlement_failure_reason[i] = np.random.choice(
                    ['Insufficient Funds', 'Counterparty Short to Deliver'], p=[0.7, 0.3]
                )
            elif trade_type[i] == 'Sell':
                settlement_failure_reason[i] = np.random.choice(
                    ['Insufficient Funds at the Counterparty', 'Client Short to Deliver'], p=[0.6, 0.4]
                )
        elif liquidity[i] < 0.02:  # ✅ Market Closure strictly when liquidity < 0.02
            settlement_failure_reason[i] = 'Market Closure'
        elif market_volatility[i] > 0.5:  # Technical Error only when volatility is high
            settlement_failure_reason[i] = 'Technical Error'
        else:
            settlement_failure_reason[i] = np.random.choice(failure_reasons_ordered, p=failure_weights)

# ✅ Ensure that "Market Closure" is assigned only when liquidity < 0.02
for i in range(num_records):
    if settlement_failure_reason[i] == "Market Closure" and liquidity[i] >= 0.02:
        settlement_failure_reason[i] = np.random.choice(failure_reasons_ordered, p=failure_weights)

# ✅ Ensure that "Insufficient Funds" and related reasons are only assigned where trade_volume > 80,000
for i in range(num_records):
    if settlement_failure_reason[i] in [
        'Insufficient Funds', 'Counterparty Short to Deliver', 'Insufficient Funds at the Counterparty', 'Client Short to Deliver'
    ]:
        if trade_volume[i] <= 80000:  # If wrongly assigned, correct it
            settlement_failure_reason[i] = np.random.choice(failure_reasons_ordered, p=failure_weights)

# ✅ Ensure that "Technical Error" is assigned only when market_volatility > 0.5
for i in range(num_records):
    if settlement_failure_reason[i] == "Technical Error" and market_volatility[i] <= 0.5:
        settlement_failure_reason[i] = np.random.choice(failure_reasons_ordered, p=failure_weights)

# Add Trade Matched or Not column
trade_matched = np.random.choice(['Matched', 'Not Matched'], size=num_records, p=[0.95, 0.05])

# Generate Expected Settlement Dates within the past year
expected_settlement_dates = pd.to_datetime(
    [datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365)) for _ in range(num_records)]
)

# Generate Actual Settlement Dates based on Settlement Status
actual_settlement_dates = [
    expected_settlement_dates[i] if settlement_status[i] == 'Success' else 
    expected_settlement_dates[i] + timedelta(days=np.random.randint(1, 6))
    for i in range(num_records)
]

# Calculate Days Difference
days_difference = [
    (actual_settlement_dates[i] - expected_settlement_dates[i]).days
    for i in range(num_records)
]

# Combine data into a DataFrame
data = {
    'Trade ID': trade_id,
    'Trade Type': trade_type,
    'Instrument Type': instrument_type,
    'Trade Value': trade_value,
    'Trade Volume': trade_volume,
    'Counterparty ID': counterparty_ids,
    'Counterparty Risk Score': counterparty_risk_score,
    'Counterparty Failures': counterparty_failures,
    'Settlement Status': settlement_status,
    'Settlement Failure Reason': settlement_failure_reason,
    'Settlement Duration': settlement_duration,
    'Market Volatility': market_volatility,
    'Liquidity': liquidity,
    'Processing Time': processing_time,
    'Manual Intervention': manual_intervention,
    'Time to Settle': time_to_settle,
    'Counterparty Failure Rate': counterparty_failure_rate,
    'Currency': currencies,
    'Settlement Type': settlement_type,
    'Client ID': client_ids,
    'ISIN': isins,
    'Trade Matched or Not': trade_matched,
    'Expected Settlement Date': expected_settlement_dates,
    'Actual Settlement Date': actual_settlement_dates,
    'Days Difference': days_difference
}

df = pd.DataFrame(data)

# Save dataset to CSV
df.to_csv('TradeSettlementDetails.csv', index=False)

print("Dataset created and saved as 'TradeSettlementDetails.csv'.")


Dataset created and saved as 'TradeSettlementDetails.csv'.


In [2]:
import pandas as pd
dataset=pd.read_csv('TradeSettlementDetails.csv')
dataset

,Trade ID,Trade Type,Instrument Type,Trade Value,Trade Volume,Counterparty ID,Counterparty Risk Score,Counterparty Failures,Settlement Status,Settlement Failure Reason,...,Time to Settle,Counterparty Failure Rate,Currency,Settlement Type,Client ID,ISIN,Trade Matched or Not,Expected Settlement Date,Actual Settlement Date,Days Difference
0,T00001,Buy,Equity,9.403941e+05,21276,CP4,0.408282,0,Fail,Market Closure,...,54.479502,0.00,EUR,FoP,CL26,INE0000432,Matched,2024-06-10,2024-06-15,5
1,T00002,Sell,Bond,9.116605e+05,93961,CP48,0.930576,9,Fail,Insufficient Funds at the Counterparty,...,60.979822,0.09,INR,FoP,CL20,INE000006Z,Not Matched,2024-03-04,2024-03-07,3
2,T00003,Buy,Equity,1.317470e+05,54387,CP48,0.930576,9,Fail,Technical Error,...,42.890503,0.09,EUR,FoP,CL14,INE000037F,Matched,2024-03-20,2024-03-22,2
3,T00004,Buy,Bond,1.202537e+06,80508,CP49,0.654659,0,Success,NaN,...,50.715000,0.00,INR,AoP,CL11,INE000009S,Matched,2024-12-02,2024-12-02,0
4,T00005,Buy,Derivative,1.630642e+05,65986,CP44,0.794073,3,Success,NaN,...,25.950168,0.03,EUR,AoP,CL42,INE000002E,Matched,2024-04-18,2024-04-18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14895,T14896,Sell,Bond,3.466039e+05,13587,CP45,0.435527,2,Fail,Quantity Difference,...,44.847009,0.02,USD,FoP,CL21,INE000013O,Matched,2024-02-19,2024-02-23,4
14896,T14897,Buy,Derivative,5.957277e+05,11223,CP36,0.971204,2,Success,NaN,...,49.328888,0.02,GBP,AoP,CL18,INE000039A,Matched,2024-02-14,2024-02-14,0
14897,T14898,Buy,Bond,1.505693e+06,93494,CP8,0.743202,7,Success,NaN,...,38.517444,0.07,EUR,AoP,CL12,INE000013O,Matched,2024-04-11,2024-04-11,0
14898,T14899,Buy,Derivative,9.474777e+05,96128,CP27,0.785000,9,Fail,Insufficient Funds,...,74.918756,0.09,INR,AoP,CL20,INE000023Y,Matched,2024-04-27,2024-05-02,5


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14900 entries, 0 to 14899
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Trade ID                   14900 non-null  object 
 1   Trade Type                 14900 non-null  object 
 2   Instrument Type            14900 non-null  object 
 3   Trade Value                14900 non-null  float64
 4   Trade Volume               14900 non-null  int64  
 5   Counterparty ID            14900 non-null  object 
 6   Counterparty Risk Score    14900 non-null  float64
 7   Counterparty Failures      14900 non-null  int64  
 8   Settlement Status          14900 non-null  object 
 9   Settlement Failure Reason  6750 non-null   object 
 10  Settlement Duration        14900 non-null  float64
 11  Market Volatility          14900 non-null  float64
 12  Liquidity                  14900 non-null  float64
 13  Processing Time            14900 non-null  int

In [4]:
dataset['Settlement Status'].value_counts()

Settlement Status
Success    8150
Fail       6750
Name: count, dtype: int64

In [12]:
dataset['Settlement Failure Reason'].value_counts()

Settlement Failure Reason
Technical Error                           1619
Account Difference                         794
Market Closure                             590
Invalid Counterparty                       554
Insufficient Funds                         452
Currency Difference                        447
Quantity Difference                        403
Insufficient Funds at the Counterparty     399
Documentation Issue                        394
Client Instruction Missing                 332
Issues at the CSD                          316
Client Short to Deliver                    275
Counterparty Short to Deliver              175
Name: count, dtype: int64

In [5]:
dataset.nunique()

Trade ID                     14900
Trade Type                       2
Instrument Type                  3
Trade Value                  14900
Trade Volume                 13895
Counterparty ID                 49
Counterparty Risk Score         49
Counterparty Failures           10
Settlement Status                2
Settlement Failure Reason       13
Settlement Duration          14900
Market Volatility            14900
Liquidity                    14900
Processing Time                 59
Manual Intervention              2
Time to Settle               14900
Counterparty Failure Rate       10
Currency                         5
Settlement Type                  2
Client ID                       49
ISIN                            50
Trade Matched or Not             2
Expected Settlement Date       365
Actual Settlement Date         370
Days Difference                  6
dtype: int64